# Evaluation criteria

The goal of this assignment is to get a view on your hands-on "data engineering" skills.  
At our company, our data scientists and engineers collaborate on projects.  
Your main focus will be creating performant & robust data flows.  
For a take-home-assignment, we cannot grant you access to our infrastructure.  
The assignement below measures your proficiency in general programming, data science & engineering tasks using python.  
Completion should not take more than half a day.

**We expect you to be proficient in:**
 * SQL queries (Sybase IQ system)
 * ETL flows (In collaboration with existing teams)
 * General python to glue it all together
 * Python data science ecosystem (Pandas + SKlearn)
 
**In this exercise we expect you to demonstrate your ability to / knowledge of:**
 * Building a data science runtime
 * PEP8 / Google python styleguide
 * Efficiently getting the job done
 * Choose meaningfull names for variables & functions
 * Writing maintainable code (yes, you might need to document some steps)
 * Help a data scientist present interactive results.
 * Offer predictions via REST api

# Setting-up a data science workspace

We allow you full freedom in setting up a data science runtime.  
The main objective is having a runtime where you can run this notebook and the code you will develop.  
You can choose for a local setup on your pc, or even a cloud setup if you're up for it.   

**In your environment, you will need things for:**
 * https request
 * python3 (not python2 !!)
 * (geo)pandas
 * interactive maps (e.g. folium, altair, ...)
 * REST apis
 
**Deliverables we expect**:
 * notebook with the completed assignment
 * list of packages for your runtime (e.g. yml or txt file)
 * evidence of a working API endpoint

# Importing packages

We would like you to put all your import statements here, together in 1 place.  
Before submitting, please make sure you remove any unused imports :-)  

In [50]:
# All the imports

import pandas as pd
import requests
import numpy as np
import geopandas as gpd
import folium
import math
import unittest
import joblib
import lightgbm

# Data ingestion exercises

## Getting store location data from an API

**Goal:** Obtain a pandas dataframe  
**Hint:** You will need to normalise/flatten the json, because it contains multiple levels  
**API call:** https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places  

In [70]:
#This function triggers the rest api url and returns the json response 
def get_clp_places(url):
    if len(url.strip())==0:
        raise ValueError("url should not be empty")
    return requests.get(url).json()

In [73]:
## Calling the get_clp_places function to fetch all the stores locations and stroing in a constant
CLP_PLACES_JSON = get_clp_places("https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places")

First approach where, we try to normalize the placeSearchOpeningHours sparately so that the dataframe of store locations do not have repetative values for the brach.

In [77]:

df_clp = pd.json_normalize(CLP_PLACES_JSON)
#Here we delete the column with nested list 
del(df_clp['placeSearchOpeningHours'])
df_clp.head(5)

,placeId,commercialName,branchId,sourceStatus,sellingPartners,handoverServices,moreInfoUrl,routeUrl,isActive,ensign.id,...,placeType.longName,placeType.placeTypeDescription,geoCoordinates.latitude,geoCoordinates.longitude,address.streetName,address.houseNumber,address.postalcode,address.cityName,address.countryName,address.countryCode
0,902,AALST (COLRUYT),4156,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.933074,4.0538972",True,8,...,Winkel,Winkel,50.933074,4.053897,BRUSSELSE STEENWEG,41,9300,AALST,België,BE
1,946,AALTER (COLRUYT),4218,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=51.0784761,3.450...",True,8,...,Winkel,Winkel,51.078476,3.450013,LOSTRAAT,66,9880,AALTER,België,BE
2,950,AARSCHOT (COLRUYT),4222,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.9760369,4.811...",True,8,...,Winkel,Winkel,50.976037,4.811097,LEUVENSESTEENWEG,241,3200,AARSCHOT,België,BE
3,886,ALSEMBERG (COLRUYT),4138,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.7415212,4.336719",True,8,...,Winkel,Winkel,50.741521,4.336719,BRUSSELSESTEENWEG,19,1652,ALSEMBERG,België,BE
4,783,AMAY (COLRUYT),3853,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.5599284,5.306...",True,8,...,Winkel,Winkel,50.559928,5.306195,CHAUSSEE DE TONGRES,247,4540,AMAY,België,BE


In [78]:
# Here we  normalize the nested list of placeSearchOpeningHours along with placeId
df_Opening_hours = pd.json_normalize(
    CLP_PLACES_JSON, "placeSearchOpeningHours", "placeId"
)
df_Opening_hours.head(5)

,date,opens,closes,isToday,isOpenForTheDay,placeId
0,14-07-2023,830,2100,True,True,902
1,15-07-2023,830,2000,False,True,902
2,14-07-2023,830,2100,True,True,946
3,15-07-2023,830,2000,False,True,946
4,14-07-2023,830,2100,True,True,950


In [79]:
# Here if we want the merged data, we can merge it on place id
merged_df_clp = pd.merge(df_clp, df_Opening_hours, on='placeId', how='inner')
merged_df_clp.head(5)

,placeId,commercialName,branchId,sourceStatus,sellingPartners,handoverServices,moreInfoUrl,routeUrl,isActive,ensign.id,...,address.houseNumber,address.postalcode,address.cityName,address.countryName,address.countryCode,date,opens,closes,isToday,isOpenForTheDay
0,902,AALST (COLRUYT),4156,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.933074,4.0538972",True,8,...,41,9300,AALST,België,BE,14-07-2023,830,2100,True,True
1,902,AALST (COLRUYT),4156,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.933074,4.0538972",True,8,...,41,9300,AALST,België,BE,15-07-2023,830,2000,False,True
2,946,AALTER (COLRUYT),4218,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=51.0784761,3.450...",True,8,...,66,9880,AALTER,België,BE,14-07-2023,830,2100,True,True
3,946,AALTER (COLRUYT),4218,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=51.0784761,3.450...",True,8,...,66,9880,AALTER,België,BE,15-07-2023,830,2000,False,True
4,950,AARSCHOT (COLRUYT),4222,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.9760369,4.811...",True,8,...,241,3200,AARSCHOT,België,BE,14-07-2023,830,2100,True,True


Second approach, we try to flatten the entire data in the same function. As the data has nested lists, the records are duplicated.

In [80]:
## Function which triggers the rest api and flatttens the recieved json data, including the nested lists.
df_clp_places = pd.json_normalize(CLP_PLACES_JSON, record_path=['placeSearchOpeningHours'],meta=["placeId",['ensign','id'],['ensign','name'],"commercialName","branchId","sourceStatus",["placeType",'id'],["placeType",'longName'],["placeType",'placeTypeDescription'],'sellingPartners','handoverServices',['geoCoordinates','latitude'],['geoCoordinates','longitude'],['address','streetName'],['address','houseNumber'],['address','postalcode'],['address','cityName'],['address','countryName'],['address','countryCode'],'moreInfoUrl','routeUrl','isActive']
,errors='ignore')
print(df_clp_places.head(5))
print(len(df_clp_places))

         date  opens  closes  isToday  isOpenForTheDay placeId ensign.id  \
0  14-07-2023    830    2100     True             True     902         8   
1  15-07-2023    830    2000    False             True     902         8   
2  14-07-2023    830    2100     True             True     946         8   
3  15-07-2023    830    2000    False             True     946         8   
4  14-07-2023    830    2100     True             True     950         8   

    ensign.name      commercialName branchId  ... geoCoordinates.longitude  \
0  COLR_Colruyt     AALST (COLRUYT)     4156  ...                 4.053897   
1  COLR_Colruyt     AALST (COLRUYT)     4156  ...                 4.053897   
2  COLR_Colruyt    AALTER (COLRUYT)     4218  ...                 3.450013   
3  COLR_Colruyt    AALTER (COLRUYT)     4218  ...                 3.450013   
4  COLR_Colruyt  AARSCHOT (COLRUYT)     4222  ...                 4.811097   

   address.streetName address.houseNumber address.postalcode address.cityN

### Quality checks

We would like you to add several checks on this data based on these constraints:  
 * records > 200
 * latitude between 49 and 52
 * longitude between 2 and 7
 
We dont want you to create a full blown test suite here, we're just gonna use 'asserts' from unittest

In [4]:
tc = unittest.TestCase('__init__')

In [81]:
## Here we test if the dtaframe without oepning hours has more than 200 records or not
tc.assertGreater(len(df_clp), 200, "Record count is not greater than 200.")

In [82]:
## Here we test if the datafrmae with oepning hours has more than 200 records unique brances or not
uniquebraches = df_clp_places['branchId'].nunique()
tc.assertGreater(uniquebraches, 200, "Branch count is not greater than 200.")

In [83]:
# testing if the latitides of the geolocation of the store is between 49 and 52
latitudes = df_clp['geoCoordinates.latitude']
for latitude in latitudes:
    tc.assertGreaterEqual(latitude, 49, "Latitude is below the lower limit.")
    tc.assertLessEqual(latitude, 52, "Latitude is above the upper limit.")


In [84]:
# testing if the longitude of the geolocation of the store is between 2 and 7
longitudes = df_clp['geoCoordinates.longitude']
for longitude in longitudes:
    tc.assertGreaterEqual(longitude, 2, "Longitude is below the lower limit.")
    tc.assertLessEqual(longitude, 7, "Longitude is above the upper limit.")

In [85]:
# testing if there are duplicate branch ids in the data ( considering the dataframe without opening hours)
duplicate_branch_ids = df_clp.duplicated(subset='branchId')
tc.assertFalse(duplicate_branch_ids.any(), "Duplicates branches data found.")

In [86]:
# testing is the country code of the stores is correct
valid_country_codes = ['BE', 'LU', 'FR']
invalid_country_codes = df_clp.loc[~df_clp['address.countryCode'].isin(valid_country_codes)]
tc.assertEqual(len(invalid_country_codes), 0, "Invalid country codes found in the address.")

### Feature creation

Create a new column "antwerpen" which is 1 for all stores in Antwerpen (province) and 0 for all others 

In [87]:
## first changing the type of postalcode in the data frame to int
df_clp = df_clp.astype({'address.postalcode':'int'})

# creating a new column with name antwerpen, and we add  1 to those columns where the postal code is between 2000 and 2999 ( inclusive).
# This is based on : the postal codes in belgium are segregated as per the province and those of antwerpen province fall in 20XX - 29XX
df_clp['antwerpen'] = np.where(np.logical_and(df_clp['address.postalcode']>=2000 , df_clp['address.postalcode']<3000)   , 1, 0)
df_clp["antwerpen"].value_counts()

antwerpen
0    220
1     38
Name: count, dtype: int64

## Predict used car value

A datascientist in our team made a basic model to predict car prices.  
The model was saved to disk ('lgbr_cars.model') using joblib's dump fuctionality.  
Documentation states the model is a LightGBM Regressor, trained using the sk-learn api.  

**As engineer, your task it to expose this model as REST-api.** 

First, retrieve the model via the function below.  
Change the path according to your setup.  

In [89]:
# Function to load the pretrained model from a local path
def retrieve_model(path):
    if len(path.strip())==0:
        raise ValueError("path should not be empty")
    return joblib.load(path)

# Calling the retrieve_model function using the pretrained model in local directory
lgbr_cars = retrieve_model("lgbr_cars.model")

tc.assertEqual(str(type(lgbr_cars)),"<class 'lightgbm.sklearn.LGBMRegressor'>", type(lgbr_cars))

Now you have your trained model, lets do a functional test based on the parameters below.  
You have to present the parameters in this order.  

* vehicleType: coupe
* gearbox: manuell
* powerPS: 190
* model: NaN
* kilometer: 125000
* monthOfRegistration: 5 
* fuelType: diesel
* brand: audi

Based on these parameters, you should get a predicted value of 14026.35068804
However, the model doesnt accept string inputs, see the integer encoding below:

In [126]:
# Integer encoded input to test the model by predicting
model_test_input = [[3,1,190,-1,125000,5,3,1]]

In [127]:
len(model_test_input)

1

In [129]:
#Function to predict the car price from the give input values and using the given model
def make_prediction(trained_model, single_input):
    if str(type(trained_model))!="<class 'lightgbm.sklearn.LGBMRegressor'>" :
        raise ValueError ("Please use a valid LGBMRegressor model")
    if len(single_input)>1:
        raise ValueError ("Please use a single input value")
    return (trained_model.predict(single_input))[0]

predicted_value = make_prediction(lgbr_cars, model_test_input)

tc.assertAlmostEqual(predicted_value, 14026.35, places=2)

Now you got this model up and running, we want you to **expose it as a rest api.**  
We don't expect you to set up any authentication.  
We're not looking for beautiful inputs, just make it work.  
**Building this endpoint should NOT be done in a notebook, but in proper .py file(s)**

Once its up and running, use it to predict the following input:
* [-1,1,0,118,150000,0,1,38] ==> prediction should be 13920.70

In [131]:
#Please run predictCarPrices.py using "python predictCarPrices.py"
# once the server is up , please append the following url to the host and port values generated
# /predictCarPrice?vehicleType=-1&gearbox=1&powerPS=0&model=118&kilometer=150000&monthOfRegistration=0&fuelType=1&brand=38

## Geospatial data exercise
The goal of this exercise is to read in some data from a shape file and visualize it on a map
- The map should be dynamic. I want to zoom in and out to see more interesting aspects of the map
- We want you to visualize the statistical sectors within a distance of 2KM of your home location.

Specific steps to take:
- Read in the shape file
- Transform to WGS coordinates
- Create a distance function (Haversine)
- Create variables for home_lat, home_lon and perimeter_distance
- Calculate centroid for each nis district
- Calculate the distance to home for each nis district centroid 
- Figure out which nis districts are near your home
- Create dynamic zoomable map
- Visualize the nis districts near you (centroid <2km away), on the map


In [158]:
# part 1: Reading in the data
# get this file from https://statbel.fgov.be/sites/default/files/files/opendata/Statistische%20sectoren/sh_statbel_statistical_sectors_20200101.shp.zip 
belgium_geo_df = gpd.read_file('sh_statbel_statistical_sectors_20200101/sh_statbel_statistical_sectors_20200101.shp')
 

In [159]:
# changing projection to wgs84 ( here we use epsg=3857 first to calculate centroids correctly)
belgium_geo_df.geometry = belgium_geo_df.geometry.to_crs(epsg=3857)

In [160]:
#Calculating the centroids for all the districts
belgium_geo_df['centroids'] = belgium_geo_df.geometry.centroid

In [161]:
# Let's create some variables to indicate the location of your interest 
home_lat = 50.743628
home_lon = 3.966872
perimeter_distance = 2 # km

In [166]:
""" 
Formula of Haversine used here

a = sin²(φB - φA/2) + cos φA * cos φB * sin²(λB - λA/2)
c = 2 * atan2( √a, √(1−a) )
d = R ⋅ c
φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km)
Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
"""
def haversine(lat1, lon1, lat2, lon2):
    if (lat1 < -90 or lat1 > 90 or lat2 < -90 or lat2 > 90):
        raise ValueError("Latitude should be between -90 and 90")
    if(lon1 < -180 or lon1 > 180 or lon2 < -180 or lon2 > 180):
        raise ValueError("Longitude should be between -180 and 180")
    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers
    return round(km, 3)


Next, implement some sanity checks for your distance function 

In [167]:
# Checking the distanc between North(90,0) and South pole(-90,0)
dist1 = haversine(90,0,-90,0)
tc.assertAlmostEqual(round(dist1,0), 20004, delta=11)
## Here the actual distance between two poles is 20004 as per wiki, we get a approximation error of 11 km

In [168]:
# Giving invalid inputs for lattitude to check if the excetions are raised.
tc.assertRaises(ValueError, lambda: haversine(-100,-180,100,0))

In [169]:
# Giving invalid inputs for longitude to check if the excetions are raised.
tc.assertRaises(ValueError, lambda: haversine(50, 270,60,-270))

In [170]:
# Now we chaneg the crs to EPSG:4326 so to calculate the haversine distance
belgium_geo_df.centroids=belgium_geo_df.centroids.to_crs(epsg=4326)
belgium_geo_df.geometry=belgium_geo_df.geometry.to_crs(epsg=4326)

Calculating the distance from chosen point to all the districts in the shape file

In [171]:
## calculating distance from station to different districts using haversine
belgium_geo_df['distance_haversine'] =  belgium_geo_df.apply(lambda x: haversine(home_lat,home_lon,x['centroids'].y,x['centroids'].x), axis=1)

In [172]:
## filter the districts which are near to the station i.e. within 2 Km radius
nearby_districts_new = belgium_geo_df[belgium_geo_df['distance_haversine']<=perimeter_distance]
nearby_districts_new

,CS01012020,T_SEC_NL,T_SEC_FR,T_SEC_DE,C_NIS6,T_NIS6_NL,T_NIS6_FR,CNIS5_2020,T_MUN_NL,T_MUN_FR,...,T_REGIO_DE,C_COUNTRY,NUTS1,NUTS2,NUTS3,M_AREA_HA,M_PERI_M,geometry,centroids,distance_haversine
2875,23023A00-,GALMAARDEN-KERN,GALMAARDEN-KERN,GALMAARDEN-KERN,23023A,GALMAARDEN,GALMAARDEN,23023,Galmaarden,Gammerages,...,Flämische Region,BE,BE2,BE24,BE241,64.535817,4274.0,"POLYGON Z ((3.96679 50.75608 0.00000, 3.96734 ...",POINT (3.97070 50.75221),0.991
2877,23023A023,AVONDVREDE,AVONDVREDE,AVONDVREDE,23023A,GALMAARDEN,GALMAARDEN,23023,Galmaarden,Gammerages,...,Flämische Region,BE,BE2,BE24,BE241,21.277798,2557.0,"POLYGON Z ((3.97535 50.75416 0.00000, 3.97443 ...",POINT (3.97190 50.75631),1.454
2879,23023A09-,RODE,RODE,RODE,23023A,GALMAARDEN,GALMAARDEN,23023,Galmaarden,Gammerages,...,Flämische Region,BE,BE2,BE24,BE241,151.164817,6270.0,"POLYGON Z ((3.97406 50.74150 0.00000, 3.97381 ...",POINT (3.95835 50.74162),0.640
2885,23023C19-,HERHOUT,HERHOUT,HERHOUT,23023C,TOLLEMBEEK,TOLLEMBEEK,23023,Galmaarden,Gammerages,...,Flämische Region,BE,BE2,BE24,BE241,472.816784,13316.0,"POLYGON Z ((4.00689 50.72922 0.00000, 4.00680 ...",POINT (3.96795 50.73087),1.421
7438,41018S19-,EMBEKE - MEYERY,EMBEKE - MEYERY,EMBEKE - MEYERY,41018S,VIANE,VIANE,41018,Geraardsbergen,Grammont,...,Flämische Region,BE,BE2,BE23,BE231,108.138869,6750.0,"POLYGON Z ((3.94634 50.74826 0.00000, 3.94634 ...",POINT (3.94391 50.74201),1.625


Now, create a dynamical map 

In [173]:
#We create a map using folium focused on the point of interest ( Galmaarden station)
galmaarden_station = folium.Map(location = [galmaarden_station_lat,galmaarden_station_lon],zoom_start=20)
# we add the near by districts on the map
folium.GeoJson(nearby_districts_new.geometry).add_to(galmaarden_station)
#showing the map
display(galmaarden_station)